<a href="https://colab.research.google.com/github/duyqu-chan/Sikayet-Talep-Analizi/blob/master/KadikoyBel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Şikayet Verisi Analizi


---


##### Duygu Can
##### Yapay Zeka Yaz Atölyesi
##### 18 - 31 Ağustos 2019, Kadıköy IDEA


---


Bu çalışmada Kadıköy Belediyesi'ne gönderilen metinlerin şikayet, talep, vs. gibi sınıflandırılması yapılacaktır. *Kapsam* ve *Döküman Metin* kolonları arasındaki ilişki incelenecektir.

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Veriyi okuma

In [10]:
import pandas as pd
df = pd.read_excel("/content/drive/My Drive/YapayzekaAtolyesi/EEVR_D-Evrak_iletisim_detay.xlsx")
df.tail(5)

,Birim Adı,Evrak Türü,Doküman No,Kapsam,Geliş Şekli,Doküman Metin,Evrak Zaman,Evrak Sonuç Zaman,Hedef/Miat Tarihi,İşlem Süresi,Evrak Sonuç Durum,Doküman Adres Mahalle,Doküman Adres Pafta,Doküman Adres Ada,Doküman Adres Parsel,Doküman Yer Adresi,Süre Durumu,İşlem Durumu,İşlem Yapılan Konu,İşlem Yapılan Alt Konu,İşlem Yapılan Süre,İşlem Yapılan Süre Birimi,Kurum Görüşü
16150,NaN,NaN,286038,Talep,Web Form,NaN,05/07/2019 10:43:58,08/07/2019 08:56:52,2019-07-08 16:23:42,2 gün 04:03:10.00,Tamamlandı-Olumlu,HASANPAŞA MAH.,136,494,52,NaN,Süresi İçinde,İş Bitti,Bilgi Belge Talepleri ve Yazışmalar,Dilekçe cevap yazıları,3.0,İş Günü,Sayın Sabina AZİZOVA; &nbsp;Belediyemizin şu a...
16151,NaN,NaN,286095,Talep,Web Form,NaN,05/07/2019 10:49:38,08/07/2019 08:55:37,2019-07-08 16:36:26,2 gün 03:49:11.00,Tamamlandı-Olumlu,HASANPAŞA MAH.,136,494,52,NaN,Süresi İçinde,İş Bitti,Bilgi Belge Talepleri ve Yazışmalar,Dilekçe cevap yazıları,3.0,İş Günü,Sayın Dılbar ALIYEVA; &nbsp;Belediyemizin şu a...
16152,NaN,NaN,289454,Talep,Web Form,"Merhaba, ben Peyzaj Mimarlığı bölümünden yeni ...",08/07/2019 10:04:10,08/07/2019 10:18:14,2019-07-09 11:30:00,0 gün 01:48:14.00,Tamamlandı-Olumlu,HASANPAŞA MAH.,NaN,NaN,NaN,"CUMHURİYET MAH., BAYRAKLI SOK., FATİH BEY APAR...",Süresi İçinde,İş Bitti,Bilgi Belge Talepleri ve Yazışmalar,Dilekçe cevap yazıları,3.0,İş Günü,Sayın Büşra IŞIK; &nbsp;Belediyemizin şu an iç...
16153,NaN,NaN,295153,Talep,Web Form,"İyi günler, iyi çalışmalar diliyorum. Aktif bi...",16/07/2019 08:39:06,16/07/2019 15:47:29,2019-07-17 16:00:00,2 gün 02:47:29.00,Tamamlandı-Olumlu,* Mahalle Seçilmedi,NaN,NaN,NaN,NaN,Süresi İçinde,İş Bitti,Bilgi Belge Talepleri ve Yazışmalar,Dilekçe cevap yazıları,3.0,İş Günü,"Sayın Batuhan ERDOĞAN, &nbsp;Belediyemizin şu ..."
16154,NaN,NaN,307478,Talep,Web Form,İşsizim kadıköy belediyesinde çalışmak istiyorum.,25/07/2019 14:00:13,25/07/2019 14:06:52,2019-07-30 16:00:00,0 gün 01:06:52.00,Tamamlandı-Olumlu,* Mahalle Seçilmedi,NaN,NaN,NaN,NaN,Süresi İçinde,İş Bitti,Bilgi Belge Talepleri ve Yazışmalar,Dilekçe cevap yazıları,3.0,İş Günü,Belediyemizin şu an için memur ve kadrolu işçi...


Veri tabanımızda 16155 örnek, 23 kolon var.

In [18]:
df.shape

(16155, 23)

Veri tiplerini görüntülüyelim

In [6]:
df.dtypes

Birim Adı                            object
Evrak Türü                           object
Doküman No                            int64
Kapsam                               object
Geliş Şekli                          object
Doküman Metin                        object
Evrak Zaman                          object
Evrak Sonuç Zaman                    object
Hedef/Miat Tarihi            datetime64[ns]
İşlem Süresi                         object
Evrak Sonuç Durum                    object
Doküman Adres Mahalle                object
Doküman Adres Pafta                  object
Doküman Adres Ada                    object
Doküman Adres Parsel                 object
Doküman Yer Adresi                   object
Süre Durumu                          object
İşlem Durumu                         object
İşlem Yapılan Konu                   object
İşlem Yapılan Alt Konu               object
İşlem Yapılan Süre                  float64
İşlem Yapılan Süre Birimi            object
Kurum Görüşü                    

Kategorik kolonların özgün değerlerine bakalım (Bazıları nümerik olmalıydı).

In [9]:
for col in df.columns:
  if df[col].dtype.name == "object":
    print(col, ":", df[col].unique())
  

Birim Adı : ['Basın Yayın ve Halkla İlişkiler Müdürlüğü' nan 'Bilgi İşlem Müdürlüğü'
 'Destek Hizmetleri Müdürlüğü' 'Emlak ve İstimlak Müdürlüğü'
 'Fen İşleri Müdürlüğü' 'Gençlik ve Spor Hizmetleri Müdürlüğü'
 'Kentsel Tasarım Müdürlüğü' 'Kültür ve Sosyal İşler Müdürlüğü'
 'Mali Hizmetler Müdürlüğü' 'Park ve Bahçeler Müdürlüğü'
 'Plan ve Proje Müdürlüğü' 'Ruhsat ve Denetim Müdürlüğü'
 'Sağlık İşleri Müdürlüğü' 'Sosyal Destek Hizmetleri Müdürlüğü'
 'Temizlik İşleri Müdürlüğü' 'Ulaşım Hizmetleri Müdürlüğü'
 'Yapı Kontrol Müdürlüğü' 'Yazı İşleri Müdürlüğü' 'Zabıta Müdürlüğü'
 'Çevre Koruma ve Kontrol Müdürlüğü' 'Özel Kalem Müdürlüğü'
 'İmar ve Şehircilik Müdürlüğü' 'İnsan Kaynakları ve Eğitim Müdürlüğü']
Evrak Türü : ['Gelen Evrak' nan]
Kapsam : ['Talep' nan 'Şikayet' 'İhbar' 'Öneri' 'Teşekkür']
Geliş Şekli : ['Telefon' nan 'Web Form']
Doküman Metin : ['İlgili adrese ikramlık talebinde bulunulmuştur.Bilgilerinize.' nan
 'İlgili adrese ikramlık talep edilmiştir.Bilgilerinize. ' ...
 'Merha

## Kayıp Veri Temizleme

*Döküman Metin* kolonunun ~%15'i boş, bunlar temizlenmeli.

In [25]:
sum(df["Doküman Metin"].isna())/len(df)*100

15.122253172392448

In [26]:
import numpy as np
#df2 = df.copy()
df = df.dropna(subset=['Doküman Metin'])
#df = df[df['Doküman Metin'] != np.nan]
df.shape

(13712, 23)

In [21]:
sum(df["Doküman Metin"].isnull())/len(df)*100

15.122253172392448

Kapsam kolonunun ~%14'ü boş, bunlar da temizlenmeli.

In [19]:
sum(df["Kapsam"].isnull())/len(df)*100

14.992262457443516

In [27]:
df = df.dropna(subset=['Kapsam'])
df.shape

(13712, 23)

## Ön-işleme
 ### Büyük/Küçük Harf
 
 *Döküman Metin* kolonunundaki girdilerin hepsini küçük harf yapalım çünkü *nltk* kütüphanesindeki dolgu sözcükleri küçük harfle yazılmış.
 

In [44]:
df['Doküman Metin'] = df['Doküman Metin'].str.lower()
df['Doküman Metin'].head()

0    i̇lgili adrese ikramlık talebinde bulunulmuştu...
2    i̇lgili adrese ikramlık talep edilmiştir.bilgi...
4    i̇lgili adres için ikramlık talep edildi.bilgi...
6    fikirtepe mah.kader sok. no : 29/3 ikramlık ta...
8    05353074596 telefonun numarasının sms sistemin...
Name: Doküman Metin, dtype: object

### Son Sözler...

Metin içeriğinin son kelimeleri incelendiğinde dilekçelerin "bilginize", "bilgilerinize" bittiği gözlemlendi. Dolgu sözcüğü olarak ele alınabilirler. Bazen bu kelimeler basit bir yazım hatası yüzünden önceki kelimeye bitişik yazılmıştır. "." karakterleri " " (boşluk) ile değiştirilerek, bu kelimeler dolgu kelimesi listesine eklenebilir.


In [67]:
df['Doküman Metin'].str.rsplit(' ',1).str[1].head(10)

0     bulunulmuştur.bilgilerinize.
2                                 
4                edildi.bilginize.
6                edildi.bilginize.
8                   bilgilerinize.
9          bulunuldu.bi̇lgi̇ni̇ze.
11         bulunuldu.bi̇lgi̇ni̇ze.
12                  bilgilerinize.
13     edilmektedir.bilgilerinize.
15      edi̇ldi̇.bi̇lgi̇leri̇ni̇ze
Name: Doküman Metin, dtype: object

In [81]:
df['Doküman Metin'].str.rsplit(' ',1).str[1].str.rsplit('.').str[1].head(100)

0          bilgilerinize
2                    NaN
4              bilginize
6              bilginize
8                       
9           bi̇lgi̇ni̇ze
11          bi̇lgi̇ni̇ze
12                      
13         bilgilerinize
15     bi̇lgi̇leri̇ni̇ze
17          bi̇lgi̇ni̇ze
18             bilginize
20             bilginize
21             bilginize
22                      
23                      
24         bilgilerinize
26                      
27                   NaN
29                      
30                      
31                   NaN
33                   NaN
34         bilgilerinize
36             bilginize
37                      
38                      
40                      
42                      
44                      
             ...        
96                      
97         bilgilerinize
98                      
99         bilgilerinize
101                     
102                     
103                     
104                     
105                     


## Öznitelik Çıkarımı

### Kelime Sayısı

In [45]:
df['kelime_sayisi'] = df['Doküman Metin'].apply(lambda x: len(str(x).split(" ")))
df[['Doküman Metin','kelime_sayisi']].head()

,Doküman Metin,kelime_sayisi
0,i̇lgili adrese ikramlık talebinde bulunulmuştu...,5
2,i̇lgili adrese ikramlık talep edilmiştir.bilgi...,6
4,i̇lgili adres için ikramlık talep edildi.bilgi...,6
6,fikirtepe mah.kader sok. no : 29/3 ikramlık ta...,9
8,05353074596 telefonun numarasının sms sistemin...,9


### Karakter Sayısı

Metin uzunluklarından karakter sayısı çıkarılmıştır. Boşluk karakterleri de sayılmıştır.

In [46]:
df['karakter_sayisi'] = df['Doküman Metin'].str.len() ## this also includes spaces
df[['Doküman Metin','karakter_sayisi']].head()

,Doküman Metin,karakter_sayisi
0,i̇lgili adrese ikramlık talebinde bulunulmuştu...,62
2,i̇lgili adrese ikramlık talep edilmiştir.bilgi...,56
4,i̇lgili adres için ikramlık talep edildi.bilgi...,51
6,fikirtepe mah.kader sok. no : 29/3 ikramlık ta...,67
8,05353074596 telefonun numarasının sms sistemin...,98


### Ortalama Kelime Uzunluğu

Kullanılan kelimelerinin uzunluklarının kapsama göre farklılık gösterebileceği düşünülerek her metnin ortalam kelime uzunluğu bulunmuştur.

In [32]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

df['ort_kelime'] = df['Doküman Metin'].apply(lambda x: avg_word(x))
df[['Doküman Metin','ort_kelime']].head()

,Doküman Metin,ort_kelime
0,İlgili adrese ikramlık talebinde bulunulmuştur...,11.400000
2,İlgili adrese ikramlık talep edilmiştir.Bilgil...,10.000000
4,İlgili adres için ikramlık talep edildi.Bilgin...,7.500000
6,Fikirtepe Mah.Kader Sok. No : 29/3 ikramlık ta...,6.555556
8,05353074596 telefonun numarasının SMS sistemin...,10.000000


### Dolgu Sözcüğü Sayısı

Dil işleme problemini çalışmaya başlamadan önce şikayet/talep metnindeki dolgu sözcüklerini atacağız. Bu kelimeler genelde içerik hakkında bilgilendirici olmayan bağlaçlar, hitaplar, vs.'dir. Yine de bilgi kaybetmemek için dolgu sözcükleri sayısını ayrı bir kolonda tutmayı uygun gördük. Önce *nltk* kütüphanesinin dolgu sözcüklerini indirelim:

In [36]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [94]:
from nltk.corpus import stopwords
stop = []
stop = stopwords.words('turkish')
stop.extend(["bilginize", "bilgilerinize", ".bilginize", 
             ".bilgilerinize", "bilginize.", "bilgilerinize.", 
             ".bilginize.", ".bilgilerinize.", "saygılarımla", "saygılarımla."
             ".saygılarımla", ".saygılarımla."])

df['dolgu_sayisi'] = df['Doküman Metin'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['Doküman Metin','dolgu_sayisi']].head()

,Doküman Metin,dolgu_sayisi
0,i̇lgili adrese ikramlık talebinde bulunulmuştu...,0
2,i̇lgili adrese ikramlık talep edilmiştir.bilgi...,0
4,i̇lgili adres için ikramlık talep edildi.bilgi...,1
6,fikirtepe mah.kader sok. no : 29/3 ikramlık ta...,0
8,05353074596 telefonun numarasının sms sistemin...,1


In [95]:
print(stop)
df.dolgu_sayisi.describe()

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'bilginize', 'bilgilerinize', '.bilginize', '.bilgilerinize', 'bilginize.', 'bilgilerinize.', '.bilginize.', '.bilgilerinize.', 'saygılarımla', 'saygılarımla..saygılarımla', '.saygılarımla.']


count    13712.000000
mean         2.271222
std          3.534201
min          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
max         59.000000
Name: dolgu_sayisi, dtype: float64

In [52]:
df["Doküman Metin"][110]

'bahse konu\xa0 adrese vefat eden\xa0 nihat\xa0 zorlu\xa0 için\xa0 yarın öğlene ikramlık\xa0 talep\xa0 edildi.bilgilerinize'

### Rakam Sayısı

Metinlerde geçen rakam sayısı da bir kolonda tutulmuştur.

In [99]:
df['rakam_sayisi'] = df['Doküman Metin'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['Doküman Metin', 'rakam_sayisi']].head()
df["rakam_sayisi"].unique()

array([ 0,  1,  3,  4,  2,  8,  5,  9,  6, 12, 10,  7, 15, 13, 19, 18, 28])

## Ön-işleme'ye Devam

### Noktalama İşaretlerini Kaldırma

Noktalama işaretleri içerik açısından bilgilendirici olmadıkları için kaldırılacaktır.

In [100]:
df['Doküman Metin'] = df['Doküman Metin'].str.replace('[^\w\s]','')
df['Doküman Metin'].head()

0    ilgili adrese ikramlık talebinde bulunulmuştur...
2    ilgili adrese ikramlık talep edilmiştirbilgile...
4     ilgili adres için ikramlık talep edildibilginize
6    fikirtepe mahkader sok no  293 ikramlık talep ...
8    05353074596 telefonun numarasının sms sistemin...
Name: Doküman Metin, dtype: object

### Dolgu Sözcüklerinden Kurtulma



In [103]:
df['Doküman Metin'] = df['Doküman Metin'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Doküman Metin'].head()

0    ilgili adrese ikramlık talebinde bulunulmuştur...
2    ilgili adrese ikramlık talep edilmiştirbilgile...
4          ilgili adres ikramlık talep edildibilginize
6    fikirtepe mahkader sok no 293 ikramlık talep e...
8    05353074596 telefonun numarasının sms sistemin...
Name: Doküman Metin, dtype: object

### Sık Tekrarlar

Sık tekrar eden kelimeler bilgilendirici olmayacaktır. Tüm girdilerde, en sık tekrar eden 10 kelimeyi listeleyelim. "talep" dışındaki kelimeler, içeriğin kapsamı hakkında bilgilendirici değildir.

In [105]:
freq = pd.Series(' '.join(df['Doküman Metin']).split()).value_counts()[:11]
freq

mah         5869
olduğu      4296
no          4285
sok         4129
talep       4074
ilgili      3594
bir         3455
adreste     3159
bulunan     3084
iletildi    2409
sokak       1630
dtype: int64

In [0]:
list.remove(element)

## Vektörize Etme

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = df["Doküman Metin"]

#Initialize vectorizer:
vectorizer = CountVectorizer()

# Vectorize the corpus:
X = vectorizer.fit_transform(corpus)

## Kaynaklar



1.   [Ultimate guide to deal with Text Data (using Python) – for Data Scientists and Engineers](https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/)
2.   List item

